In [1]:
%matplotlib inline
import numpy
import pylab
import SterileSearchPy as ssp


In [2]:
#An object of type "DataPaths" is used to set where we find all the data.
# make one like this:
dp=ssp.DataPaths()

#feel free to reset them. For example, in this notebook I will use custom locations for these things:
dp.mc_path="/data/user/bjones/Sterilizer/Sterilizer/test_data/"
dp.squids_files_path="/data/user/bjones/Sterilizer/Sterilizer/test_data/"

#It has these keys (tab complete dp.[tab] to see)
dp.compact_file_path  # Compact data processed and dumped to disk (.dat) is
dp.data_path          # Full scale data file (.hdf5) is
dp.domeff_spline_path # Dom efficiency correction splines are (.dat)
dp.mc_path            # Where the full scale MC files are after processEnergy (.hdf5)
dp.oversize_path      # Oversize correction function for changing oversize MC to non-oversize MC (.dat)
dp.squids_files_path  # Conventional atmospheric flux evolved by squids for various osc hypotheses are (.hdf5)
dp.prompt_squids_files_path  # as above but the prompt flux (.hdf5)
dp.xs_spline_path     # Cross section splines (.fits) 
dp.flux_splines_path  # Flux splines (.fits) are - for factorized mode only

'/home/carguelles/programs/SNOT/FluxSplines/propagated_splines/'

In [3]:
# An object of type "SteeringParams" is used to configure the fitter.
sp=ssp.SteeringParams()

# Data / MC Configuration
sp.useBurnSample       # Whether to load burnsample or full data
sp.ReadCompact         # Read compact or full data and mc
sp.xs_model_name       # Which cross section model to use
sp.modelName           # Which flux model to use
sp.oversizeFunction    # Which oversize function to use
sp.simToLoad           # Which simulation set to load

#Binning
sp.cosThbinEdge        # Cos theta binning
sp.cosThbinWidth       #  --
sp.maxCosth            #  --
sp.minCosth            #  --

sp.logEbinEdge         # Energy binning
sp.logEbinWidth        #  --
sp.maxFitEnergy        #  --
sp.minFitEnergy        #  --

#Livetime
#sp.years               # which years to load
#sp.burnSampleLivetime  # total livetime of burn sample
#sp.fullLivetime        # total livetime of full sample

#Technical stuff
sp.useFactorization    # Whether to enable oscillation factorization
sp.evalThreads         # How many threads to use for evaluating fits
sp.quiet               # Control program verbosity

False

In [4]:
# And an object of type "SterileNuParams" sets the hypothesis we will fit against.
#   Here we set it to the null hypothesis
#
snp=ssp.SterileNuParams()
snp.del14=0
snp.del24=0
snp.dm41sq=0
snp.modelId=0
snp.th14=0
snp.th24=0
snp.th34

0.0

In [ ]:
# The object that does the magic is called Sterilizer and is initialized with these parameters:
#   Note that it will take a long time to load, as it brings the data into memory.
# 
sp.ReadCompact=False
ster=ssp.Sterilizer(dp,sp,snp)


In [1]:
#Bens notes - broken.
# File path checks on all files.
# pybinding for years
# pybinding for maps